In [1]:
import os

import pandas as pd
from utils.plots_func import *
from utils.auxiliary_func import *
import config.params as params
import config.const as const
import pickle

In [6]:
data_preparation_dir = os.path.join(const.base_dir, 'figure_results', 'data_preparation')
os.makedirs(data_preparation_dir, exist_ok=True)

In [2]:
feature_type = 'lifetime'
tissue_resolution = 'patch_tissue'
models_dict = {'model_dir_1': {'dropout_rate': 0.1, 'batch_size': 32, 'model_dir': build_gnn_model_dir_name(params.model_params[1])},
'model_dir_2': {'dropout_rate': 0.1, 'batch_size': 16, 'model_dir': build_gnn_model_dir_name(params.model_params[2])},
'model_dir_3': {'dropout_rate': 0.05, 'batch_size': 32, 'model_dir':build_gnn_model_dir_name(params.model_params[3])}}

In [3]:
models_dict

{'model_dir_1': {'dropout_rate': 0.1,
  'batch_size': 32,
  'model_dir': 'batch_size_32_class_ratio_1p0_dropout_rate_0p1_epochs_50_heads_1_hidden_layers_[64, 128, 64]_lr_0p0001_model_type_GAT_output_size_1_pooling_max_test_ratio_0p1_weight_decay_0p0001'},
 'model_dir_2': {'dropout_rate': 0.1,
  'batch_size': 16,
  'model_dir': 'batch_size_16_class_ratio_1p0_dropout_rate_0p1_epochs_50_heads_1_hidden_layers_[64, 128, 64]_lr_0p0001_model_type_GAT_output_size_1_pooling_max_test_ratio_0p1_weight_decay_0p0001'},
 'model_dir_3': {'dropout_rate': 0.05,
  'batch_size': 32,
  'model_dir': 'batch_size_32_class_ratio_1p0_dropout_rate_0p05_epochs_50_heads_1_hidden_layers_[64, 128, 64]_lr_0p0001_model_type_GAT_output_size_1_pooling_max_test_ratio_0p1_weight_decay_0p0001'}}

In [5]:
results = []

for patch_size in [1000, 1500, 2000]:
    # Step 2: Walk through subdirectories to find your specific subdir (e.g., model_dir_1)
    patch_dir = f'max_distance_30/pytorch_geo/size_{patch_size}_overlap_0.75/results'
    base_directory = os.path.join(const.gnn_dir, feature_type, tissue_resolution, patch_dir)
    for k, v in models_dict.items():
        dropout_rate = v['dropout_rate']
        batch_size = v['batch_size']
        model_dir = v['model_dir']

        found_path = None

        for root, dirs, files in os.walk(base_directory):
            for dir_name in dirs:
                if dir_name == model_dir:
                    found_path = os.path.join(root, dir_name)
                    break
            if found_path:
                break

        if found_path:
            file_path = os.path.join(found_path, 'seed_results.pkl')
            if os.path.isfile(file_path):
                with open(file_path, 'rb') as f:
                    seed_results = pickle.load(f)
                    auc_scores = seed_results['auc_agg']
                    median_auc = np.median(auc_scores)
                    mean_auc = np.mean(auc_scores)
                    best_auc = np.max(auc_scores)
                    std_auc = np.std(auc_scores)

                    results.append({
                        'Patch Size': f'{patch_size}×{patch_size} pixels',
                        'Batch Size': batch_size,
                        'Dropout Rate': dropout_rate,
                        'Mean ROC-AUC': round(mean_auc, 3),
                        'STD': round(std_auc, 2),
                        'Median ROC-AUC': round(median_auc, 3),
                        'Max ROC-AUC':  round(best_auc, 3),
                        
                    })
            else:
                print(f"'seed_results.pkl' not found in {found_path}")
        else:
            print(f"Target subdirectory for model '{k}' not found.")

df_results = pd.DataFrame(results)
df_results

,Patch Size,Batch Size,Dropout Rate,Mean ROC-AUC,STD,Median ROC-AUC,Max ROC-AUC
0,1000×1000 pixels,32,0.10,0.731,0.03,0.728,0.768
1,1000×1000 pixels,16,0.10,0.712,0.02,0.716,0.749
2,1000×1000 pixels,32,0.05,0.727,0.02,0.730,0.764
3,1500×1500 pixels,32,0.10,0.730,0.03,0.729,0.787
4,1500×1500 pixels,16,0.10,0.721,0.03,0.713,0.771
5,1500×1500 pixels,32,0.05,0.728,0.02,0.726,0.777
6,2000×2000 pixels,32,0.10,0.705,0.03,0.704,0.774
7,2000×2000 pixels,16,0.10,0.703,0.03,0.696,0.768
8,2000×2000 pixels,32,0.05,0.717,0.03,0.713,0.780


In [7]:
gnn_hyperparameters_path = os.path.join(data_preparation_dir, 'gnn_hyperparameters_results.xlsx')
df_results.to_excel(gnn_hyperparameters_path, index=False)